### Neural ODEs TUTORIAL
https://github.com/DiffEqML/

https://github.com/rtqichen/torchdiffeq/tree/master

https://github.com/Lightning-AI/pytorch-lightning

https://lightning.ai/docs/pytorch/stable/starter/introduction.html

https://arxiv.org/abs/2008.02389

https://arxiv.org/abs/1806.07366

In [ ]:
from torchdyn.core import NeuralODE
from torchdyn.datasets import *
from torchdyn import *

%load_ext autoreload
%autoreload 2

In [ ]:
# quick run for automated notebook validation
dry_run = False

### Generate data from a static toy dataset
We’ll be generating data from toy datasets. In torchdyn, we provide a wide range of datasets often use to benchmark and understand Neural ODEs. Here we will use the classic moons dataset and train a Neural ODE for binary classification

In [ ]:
# classification
d = ToyDataset()
X, yn = d.generate(n_samples=512, noise=1e-1, dataset_type='moons')  

In [ ]:
print(len(X))
print(X)
print(yn)

In [ ]:
import matplotlib.pyplot as plt

colors = ['orange', 'blue'] 
fig = plt.figure(figsize=(3,3))
# subplot: where do you want the plot to show; 111 start from top left
ax = fig.add_subplot(111)
for i in range(len(X)):
    ax.scatter(X[i,0], X[i,1], s=1, color=colors[yn[i].int()])
ax

Generated data can be easily loaded in the dataloader with standard PyTorch calls

In [ ]:
import torch
import torch.utils.data as data
device = torch.device("cpu") # all of this works in GPU as well :)

X_train = torch.Tensor(X).to(device)
y_train = torch.Tensor(yn.to(device))
train = data.TensorDataset(X_train, y_train)
trainloader = data.DataLoader(train, batch_size=len(X), shuffle=True)

We utilize Pytorch Lightning to handle training loops, logging and general bookkeeping. This allows torchdyn and Neural Differential Equations to have access to modern best practices for training and experiment reproducibility.

In particular, we combine modular torchdyn models with LightningModules via a Learner class:

In [ ]:
import torch.nn as nn
import pytorch_lightning as pl

class Learner(pl.LightningModule):
    def __init__(self, t_span:torch.Tensor, model:nn.Module):
        super().__init__()
        self.model, self.t_span = model, t_span

    # one entire forward pass of the NN
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        print(x)
        print(y)
        t_eval, y_hat = self.model(x, self.t_span)
        y_hat = y_hat[-1] # select last point of solution trajectory
        loss = nn.CrossEntropyLoss()(y_hat, y)
        return {'loss': loss}   
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.01)

    def train_dataloader(self):
        return trainloader

In [ ]:
t_span = torch.linspace(0,1,100)
t_eval, trajectory = model(X_train, t_span)
trajectory = trajectory.detach().cpu()

In [ ]:
f = nn.Sequential(
        nn.Linear(2, 16),
        nn.Tanh(),
        nn.Linear(16, 2)                       
    )

model = NeuralODE(f, sensitivity='adjoint', solver='rk4', solver_adjoint='dopri5', atol_adjoint=1e-4, rtol_adjoint=1e-4).to(device)

learn = Learner(t_span, model)
if dry_run: trainer = pl.Trainer(min_epochs=1, max_epochs=1)
else: trainer = pl.Trainer(min_epochs=200, max_epochs=300)
trainer.fit(learn)

In [ ]:
color=['orange', 'blue']

fig = plt.figure(figsize=(10,2))
ax0 = fig.add_subplot(121)
ax1 = fig.add_subplot(122)
for i in range(500):
    ax0.plot(t_span, trajectory[:,i,0], color=color[int(yn[i])], alpha=.1);
    ax1.plot(t_span, trajectory[:,i,1], color=color[int(yn[i])], alpha=.1);
ax0.set_xlabel(r"$t$ [Depth]") ; ax0.set_ylabel(r"$h_0(t)$")
ax1.set_xlabel(r"$t$ [Depth]") ; ax1.set_ylabel(r"$z_1(t)$")
ax0.set_title("Dimension 0") ; ax1.set_title("Dimension 1")

In [ ]:
for i in range(len(trajectory[0])):
    print("x: " + str(trajectory[0][i]) + " y: " + str(trajectory[-1][i]))

In [ ]:
x = torch.Tensor([[a] for a in torch.linspace(-20, 20, 81)])
y = torch.exp(x.flatten())

In [ ]:
class Exponential_Learner(pl.LightningModule):
    def __init__(self, t_span:torch.Tensor, model:nn.Module):
        super().__init__()
        self.model, self.t_span = model, t_span

    # one entire forward pass of the NN
    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch      
        t_eval, y_hat = self.model(x, self.t_span)
        y_hat = y_hat[-1] # select last point of solution trajectory
        loss = nn.MSELoss()(y_hat, y)
        return {'loss': loss}   
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.01)

    def train_dataloader(self):
        return trainloader

In [ ]:
x_train = torch.Tensor(x).to(device)
y_train = torch.Tensor(y.to(device))
train = data.TensorDataset(x_train, y_train)
trainloader = data.DataLoader(train, batch_size=len(x), shuffle=True)

In [ ]:
f = nn.Sequential(
        nn.Linear(1, 16),
        nn.ELU(),
        nn.Linear(16, 1)                       
    )

modelE = NeuralODE(f, sensitivity='adjoint', solver='rk4', solver_adjoint='dopri5', atol_adjoint=1e-4, rtol_adjoint=1e-4).to(device)

In [ ]:
t_span = torch.linspace(0,1,100)
learnE = Exponential_Learner(t_span, modelE)
if dry_run: trainer = pl.Trainer(min_epochs=1, max_epochs=1)
else: trainer = pl.Trainer(min_epochs=200, max_epochs=300)
trainer.fit(learnE)

In [ ]:
t_eval, trajectory = modelE(x_train, t_span)
trajectory = trajectory.detach().cpu()

In [ ]:
fig = plt.figure(figsize=(20,10))
ax0 = fig.add_subplot(111)
for i in range(81):
    ax0.plot(t_span, trajectory[:,i], color='black', alpha=.1);

In [ ]:
fig = plt.figure(figsize=(5,5))
ax0 = fig.add_subplot(111)
ax0.plot(x, trajectory[-1,:], color='black', alpha=.1);

In [ ]:
fig = plt.figure(figsize=(5,5))
ax0 = fig.add_subplot(111)
ax0.plot(x, y, color='black', alpha=.1);